In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# University of California Admissions by Source School, 2016 to 2022
`universityofcalifornia.edu-admissions-1.1.2` Last Update: 2023-04-25T03:32:13

_These tables present information on UC’s Applicants, Admits, and Enrollees. Freshman applicant counts are presented for fall and transfer counts are presented for the full academic year.  Mean high school and transfer GPA are presented for freshman and transfer applicants, admits, and enrollees respectively by source school._


This data package presents data scraped from a Tableau vizualization on the
Univerity of California's data system, for the admissions to UC Campuses by the
source high school. The dataset provides the number of students that applied from each high school to each UC Campus, as well as the number admitted and enrolled. 


The UC data was collected by manually "scraping" the Tableau vizualiztion; each
combination of UC campus and year was selected and downloaded to a file, and
these files were combined into a single dataset. Then, the UC records were
matched to California Department of Education records using a fuzzy match on
the names. Only about 83% of the records could be relabily matched, with a
score of .8 or greater. See the ``matches`` file for all of the matches and
their scores.
## Documentation Links

* [Data Viz Page](https://www.universityofcalifornia.edu/about-us/information-center/admissions-source-school) THe web page that hosts the datavizualization

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* **[uc_admissions](notebooks/build.ipynb#df)**. UC admissions data, by source school, for multiple years
* **[matches](notebooks/build.ipynb#match_df)**. Control file for the name matching between the UC data and CDE records.

## References
<ul><li> <strong>public_schools</string>, <em>metapack+http://library.metatab.org/cde.ca.gov-schools-2.1.1.csv</em>. California public schools</li><ul>

In [3]:
from pathlib import Path


name_rows = []

for f in (Path(pkg.path).parent/'data'/'FR ENR').glob('*.csv'):
    t =f.stem.replace('FR ENR','');
    year = int(t.split()[-1])
    school = t.replace(str(year),'')
    name_rows.append({'year':year, 'campus':school, 'path':f})
    
paths_df = pd.DataFrame(name_rows)

# Check that all of the records are there. 
assert all(paths_df.groupby('year').count().campus == 10)
assert all(paths_df.groupby('campus').count().year == 7)

In [4]:
frames = []


for idx, r in paths_df.iterrows():
    
    df = pd.read_csv(r.path, delimiter='\t', encoding='utf-16').assign(year=r.year, campus=r.campus)

        
    frames.append(df)
    
t = pd.concat(frames).reset_index(drop=True)
t = t.drop(columns=['Calculation1'])

cols = ['school', 'city', 'region', 'count', 'all', 'black', 'aian', 'hisp', 'nhpi', 
           'asian', 'white', 'na', 'international', 'year', 'campus']


t = t.rename(columns=dict(zip(t.columns, cols)))


In [8]:
uc_df = t[['year', 'school',  'campus', 'city', 'region', 'count', 'all', 'black', 'aian', 'hisp', 'nhpi', 
              'asian', 'white', 'na', 'international']].copy()

def strip_words(v):
    import re
    
    v = v.lower()
    v = re.sub('school$','',v)
    v = v.replace('high','').replace('senior','').replace('hs','')
    v = re.sub('\s+',' ',v)
    
    return v

uc_df['match_school'] = uc_df.school.apply(strip_words)

uc_df.sample(10)

,year,school,campus,city,region,count,all,black,aian,hisp,nhpi,asian,white,na,international,match_school
62282,2019,LIVERMORE HIGH SCHOOL,Los Angeles,Livermore,Alameda,Enr,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,livermore
121356,2022,BANNING HIGH SCHOOL,Santa Barbara,Banning,Riverside,App,12.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,banning
73807,2019,SEASIDE HIGH SCHOOL,Davis,Seaside,Monterey,Adm,9.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,seaside
149591,2022,LIFE ACADEMY HIGH SCHOOL,Berkeley,Oakland,Alameda,Enr,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,life academy
130869,2016,MONTGOMERY HIGH SCHOOL,Davis,Santa Rosa,Sonoma,App,31.0,NaN,NaN,7.0,NaN,NaN,14.0,NaN,NaN,montgomery
82871,2016,EVEREST PUBLIC HIGH SCHOOL,Santa Barbara,Redwood City,San Mateo,Enr,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,everest public
7398,2021,SANTA BARBARA SENIOR HIGH SCHO,Los Angeles,Santa Barbara,Santa Barbara,App,109.0,NaN,NaN,37.0,NaN,8.0,51.0,6.0,NaN,santa barbara scho
145911,2019,JOHN F KENNEDY HIGH SCHOOL,Universitywide,Granada Hills,Los Angeles,App,93.0,NaN,NaN,73.0,NaN,5.0,6.0,NaN,NaN,john f kennedy
80026,2018,GOLDEN VALLEY HIGH SCHOOL,Davis,Bakersfield,Kern,Adm,9.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,golden valley
152945,2016,SAN LORENZO HIGH SCHOOL,Santa Cruz,San Lorenzo,Alameda,Enr,6.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,san lorenzo


In [9]:
sch_pkg = mp.open_package('http://library.metatab.org/cde.ca.gov-schools-2.1.1.csv')

# Create Dataframes
public_schools_df = sch_pkg.resource('public_schools').dataframe()
pubhs_df = public_schools_df[public_schools_df.eilcode == "HS"]


In [10]:
# Build a dict that maps names to CDS codes
from thefuzz import process
from thefuzz import fuzz

match_map = {}

for county, g in pubhs_df.groupby('county'):
    match_map[county.lower()] = {}
    for idx, r in g.iterrows():
        
        school = strip_words(r.school.lower())
        
        match_map[county.lower()][school] = r.cdscode

In [11]:
def find_match(region, school):
    
    region = region.lower()
    school == school.lower()
    
    name_map = match_map[region]
    
    match  = process.extractOne(school, name_map.keys(), scorer=fuzz.partial_token_sort_ratio)
    
    return {
        'score': match[1],
        'qscore': match[1]//10,
        'match_school': school,
        'match': match[0],
        'region': region,
        'cdscode': name_map[match[0]]
    }

In [12]:
# Use fuzzy matching to find the CDScode from the UC high schools names
from tqdm.auto import tqdm

matches = []
for idx, r in tqdm(list(uc_df[['region','match_school']].drop_duplicates().iterrows())):
    matches.append(find_match(r.region,r.match_school))
    
match_df = pd.DataFrame(matches)

match_df.head()

  0%|          | 0/1675 [00:00<?, ?it/s]

,score,qscore,match_school,match,region,cdscode
0,80,8,a b miller,fontana a. b. miller,san bernardino,36677103630555
1,100,10,abraham lincoln,lincoln (abraham),san francisco,38684783833241
2,100,10,abraham lincoln,abraham lincoln,santa clara,43696664333795
3,71,7,academia avance charter,arcadia,los angeles,19642611930288
4,81,8,academic leadership community,animo leadership,los angeles,19647091996313


In [13]:
# Report on what percentage of the matches will be used. 80% look like a good cut off, but there are
# certainly a few false positivies & negatives. 
match_df.groupby(match_df.score >= 80).qscore.count()/len(match_df)

score
False    0.172537
True     0.827463
Name: qscore, dtype: float64

In [16]:
t = uc_df.merge(match_df, on='match_school')
t = t.rename(columns={'school':'uc_school'}).merge(pubhs_df, on='cdscode')

cols = ['year', 'cdscode', 'ncesdist', 'ncesschool',  'school', 'uc_school', 'campus', 
        'city_x', 'city_y', 'region_x', 'county', 'district',
        'count', 'all', 'black', 'aian', 'hisp', 'nhpi', 'asian', 'white', 'na', 'international',  
        'region_y',  'charter', 'virtual', 'magnet', 'eilname',  'gsserved',
         'zip', 'latitude', 'longitude', 'score' ]

df = t[cols].copy().rename(columns={'city_x':'uc_city','city_y':'cde_city', 'region_x':'county'})

df.sample(10).head()

,year,cdscode,ncesdist,ncesschool,school,uc_school,campus,uc_city,cde_city,county,...,region_y,charter,virtual,magnet,eilname,gsserved,zip,latitude,longitude,score
197765,2018,39685930116376,0623610,12065,Lathrop High,LATHROP HIGH SCHOOL,Los Angeles,Lathrop,Lathrop,San Joaquin,...,san joaquin,N,N,N,High School,9-12,95330-9770,37.826445,-121.28020,100
104506,2017,19649801995737,0635700,02173,Malibu High,MALIBU HIGH SCHOOL,Los Angeles,Malibu,Malibu,Los Angeles,...,los angeles,N,N,N,High School,9-12,90265-3616,34.024246,-118.82817,100
162354,2018,43104390123794,0691035,12625,Summit Public School: Tahoma,SUMMIT PUBLIC SCHOOL-TAHOMA,Davis,San Jose,San Jose,Santa Clara,...,santa clara,Y,N,N,High School,9-12,95123-2048,37.252798,-121.81957,100
71177,2021,37683383732997,0634320,05482,Hoover High,HERBERT HOOVER HIGH SCHOOL,Berkeley,Glendale,San Diego,Los Angeles,...,san diego,N,C,N,High School,9-12,92115-4312,32.755352,-117.09761,100
166546,2018,19650601938752,0639420,06576,Torrance High,TORRANCE HIGH SCHOOL,Santa Barbara,Torrance,Torrance,Los Angeles,...,los angeles,N,C,N,High School,9-12,90501-3135,33.831308,-118.32114,100


In [15]:
df.head().T.head(40)

,0,1,2,3,4
year,2022,2022,2022,2021,2021
cdscode,36677103630555,36677103630555,36677103630555,36677103630555,36677103630555
ncesdist,0613920,0613920,0613920,0613920,0613920
ncesschool,10301,10301,10301,10301,10301
school,Fontana A. B. Miller High,Fontana A. B. Miller High,Fontana A. B. Miller High,Fontana A. B. Miller High,Fontana A. B. Miller High
uc_school,A B MILLER HIGH SCHOOL,A B MILLER HIGH SCHOOL,A B MILLER HIGH SCHOOL,A B MILLER HIGH SCHOOL,A B MILLER HIGH SCHOOL
campus,Santa Cruz,Santa Cruz,Santa Cruz,Los Angeles,Los Angeles
uc_city,Fontana,Fontana,Fontana,Fontana,Fontana
cde_city,Fontana,Fontana,Fontana,Fontana,Fontana
county,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
